## Assignment-2: n-gram TF-IDF and document similarity

** Objective **
- Learn Term Frequency
- Compute document similarity

Tasks
* 1) Task 1: Dataset Preparation: Prepare the Nepali news dataset (hint: you can
obtain text from news websites, at least 20 different news of 2/3 different
categories).
Host the dataset in the public git repository.
In your notebook data should be downloaded from git or some other public
places.
No additional step should be done to get the notebook working.
You can reuse the dataset of Assignment 1 as well.
* 2) Task 2: Prepare one-gram, bi-gram, tri-gram vocabulary
* 3) Task 3: Compute TF-IDF vectors for each vocabulary
* 4) Task 4: Compute document similarity matrix (if your document size = N , this
will result in the NxN matrix) for each vocab list.
* 5) Task 5: Write your interpretation on the result of Task 4.

In [1]:
import datetime

student_rollno = 25
student_name = 'Rajendra Karki'
assignment_tag = 'MDS555-2023-Assignment-2'

In [2]:
# from checker_utils import done
def done(task):
    _date = datetime.datetime.now()
    task = task + ": " + str(_date)
    print('='*len(task), '\n', task , '\n', '='*len(task), sep='')
    pass

## Literature Review
In this NPL assignment we used the previously created nepali news data set to create the one-gram, bigram and trigram vocabualary, but before that we have done the mendatory preprocessing steps to clean, tokenize and remove stopwords from our text. After preprocessing we create the vocabualary and compute the TF-IDF vectors from those vocabualary. Lastly we create the document similarity matrix from those vectors and figure out the similarity of the words in each documents.

For this purpose we have used different library. We have used NLTK as ususal for Text Preprocessing, tokenizing and removing stopwords. Later we used nltk utility function ngrams to convert the text into n gram vocabualary. Then we use TfidfVectorizer function from sklearn / feature_extraction  to compute the TF-IDF vectors for each vocabulary and finally we create the document similarity matrix to figure out the similarities between the words in the documents.

The similarity matrix are displayed using the dataframe and for that purpose we have used the basic python library pandas. Cosine Similarity is used to prepare the document similarity matrix.

## Task 1 Dataset Preparation:
* Prepare the Nepali news dataset (hint: you can
obtain text from news websites, at least 20 different news of 2/3 different
categories).
* Host the dataset in the public git repository.
* In your notebook data should be downloaded from git or some other public
places.
* No additional step should be done to get the notebook working.
* You can reuse the dataset of Assignment 1 as well.

In [3]:
# Task 1: Dataset Preparation:
import pandas as pd
# !git clone https://github.com/rjndra/Lab.git
# # Load the dataset
# df = pd.read_csv('Lab/II/NLP/Assignment/nepali_news_dataset.csv')
# df.head()

#Getting the data from github
github_csv_url = 'https://raw.githubusercontent.com/rjndra/Lab/main/II/NLP/Assignment/nepali_news_dataset.csv'

# Read the CSV file from GitHub into a DataFrame
df = pd.read_csv(github_csv_url, encoding='utf-8')

df.head()

,Input,Category
0,"डलरको भाउ हालसम्मकै उच्च, किन बढिरहेको छ भाउ ?",business
1,१२ वर्षमा २८ हजार मेगावाट बिजुली उत्पादनको लक्ष्य,business
2,भूतप्रभावी करविरुद्ध १६ बैंक अदालतमा,business
3,तेस्रो देशका पर्यटकलाई भारतको ‘अघोषित नाकाबन्दी’,business
4,साउन र भदौमा सरकारले उठायो कुल १ खर्ब ४१ अर्ब ...,business


### Preprocessing
Do some preprocessing of the Content. You can remove the punctuation, remove special characters, tokenized the words, remove nepali stopwords and display all in the dataframe for later use.

In [4]:
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Function to clean text
def clean_text(text):
    #Lowercase the text
    #Remove punctuation and numbers, and convert to lowercase
    text = text.replace('।', '')
    text = text.replace('‘', '')
    text = text.replace('’', '')
    text = text.replace('–', '')
    text = text.lower()

    # Remove punctuation and numbers
    text = re.sub(f"[{re.escape(string.punctuation)}0-9]", '', text)

    # Join the words back into a single string
    cleaned_text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])

    return cleaned_text

# Apply the function to the 'Input' column in the DataFrame
df['Cleaned_Text'] = df['Input'].apply(clean_text)
df['Tokenized_Text'] = df['Cleaned_Text'].apply(word_tokenize)

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajendrakarki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Input,Category,Cleaned_Text,Tokenized_Text
0,"डलरको भाउ हालसम्मकै उच्च, किन बढिरहेको छ भाउ ?",business,डलरको भाउ हालसम्मकै उच्च किन बढिरहेको छ भाउ,"[डलरको, भाउ, हालसम्मकै, उच्च, किन, बढिरहेको, छ..."
1,१२ वर्षमा २८ हजार मेगावाट बिजुली उत्पादनको लक्ष्य,business,वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य,"[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्..."
2,भूतप्रभावी करविरुद्ध १६ बैंक अदालतमा,business,भूतप्रभावी करविरुद्ध बैंक अदालतमा,"[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]"
3,तेस्रो देशका पर्यटकलाई भारतको ‘अघोषित नाकाबन्दी’,business,तेस्रो देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी,"[तेस्रो, देशका, पर्यटकलाई, भारतको, अघोषित, नाक..."
4,साउन र भदौमा सरकारले उठायो कुल १ खर्ब ४१ अर्ब ...,business,साउन र भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व,"[साउन, र, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर..."


In [5]:
#import libraries
nltk.download('stopwords')
from nltk.corpus import stopwords

# Remove stop words
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('nepali'))
    return [token for token in tokens if token.lower() not in stop_words]

df['No_Stopwords'] = df['Tokenized_Text'].apply(remove_stopwords)
# Join the content without stop words and store it in a new column 'Content_No_Stopwords'
df['Content_No_Stopwords'] = df['No_Stopwords'].apply(lambda x: ' '.join(x))

df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rajendrakarki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Input,Category,Cleaned_Text,Tokenized_Text,No_Stopwords,Content_No_Stopwords
0,"डलरको भाउ हालसम्मकै उच्च, किन बढिरहेको छ भाउ ?",business,डलरको भाउ हालसम्मकै उच्च किन बढिरहेको छ भाउ,"[डलरको, भाउ, हालसम्मकै, उच्च, किन, बढिरहेको, छ...","[डलरको, भाउ, हालसम्मकै, उच्च, बढिरहेको, भाउ]",डलरको भाउ हालसम्मकै उच्च बढिरहेको भाउ
1,१२ वर्षमा २८ हजार मेगावाट बिजुली उत्पादनको लक्ष्य,business,वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य,"[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्...","[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्...",वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य
2,भूतप्रभावी करविरुद्ध १६ बैंक अदालतमा,business,भूतप्रभावी करविरुद्ध बैंक अदालतमा,"[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]","[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]",भूतप्रभावी करविरुद्ध बैंक अदालतमा
3,तेस्रो देशका पर्यटकलाई भारतको ‘अघोषित नाकाबन्दी’,business,तेस्रो देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी,"[तेस्रो, देशका, पर्यटकलाई, भारतको, अघोषित, नाक...","[देशका, पर्यटकलाई, भारतको, अघोषित, नाकाबन्दी]",देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी
4,साउन र भदौमा सरकारले उठायो कुल १ खर्ब ४१ अर्ब ...,business,साउन र भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व,"[साउन, र, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर...","[साउन, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर्ब,...",साउन भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व


In [6]:
done('Task 1')

Task 1: 2023-12-19 11:51:52.934570


##  Task 2: Prepare one-gram, bi-gram, tri-gram vocabulary

A **one-gram vocabulary** consists of individual words or tokens from your text data. In order to create the one gram vocabulary we need to first clean the text or preprocess text to remove puncuatation, unnecessary characters, then tokenize the words and remove the stop words. One gram is actually the individual words from the text.

A **bigram vocabulary** consists of pairs of consecutive words from the text data. and the **tri gram vocabulary** consists of sets of three consecutive words.  

In [8]:
#GENERATE N GRAMS FROM THE CLEANED TEXT

from nltk.util import ngrams

# Function to generate n-grams from a list of tokens
def generate_ngrams(tokens, n):
    return list(ngrams(tokens, n))

# Create one-gram, bi-gram, and tri-gram columns in the data frame
df['one_gram'] = df['No_Stopwords'].apply(lambda x: generate_ngrams(x, 1))
df['bi_gram'] = df['No_Stopwords'].apply(lambda x: generate_ngrams(x, 2))
df['tri_gram'] = df['No_Stopwords'].apply(lambda x: generate_ngrams(x, 3))

# Create one-gram, bi-gram, and tri-gram vocabularies
one_gram_vocab = set([word for sublist in df['one_gram'].tolist() for word in sublist])
bi_gram_vocab = set([word for sublist in df['bi_gram'].tolist() for word in sublist])
tri_gram_vocab = set([word for sublist in df['tri_gram'].tolist() for word in sublist])

# Print the vocabularies
print("One-gram Vocabulary:")
print(one_gram_vocab)
print("\nBi-gram Vocabulary:")
print(bi_gram_vocab)
print("\nTri-gram Vocabulary:")
print(tri_gram_vocab)


One-gram Vocabulary:
{('भागवत',), ('काम',), ('यसले',), ('बास्केटबल',), ('विद्यार्थीले',), ('रेडियन्ट',), ('क्रिकेटको',), ('उत्तिकै',), ('स्थान',), ('कार्यालयमा',), ('नाडा',), ('त्यसैले',), ('फिल्ममा',), ('सम्पन्न',), ('फिल्म',), ('खातामा',), ('डिजाइनर्स',), ('न्युकासललाई',), ('पुष्कर',), ('उक्लेको',), ('युथ',), ('दंगालको',), ('किमि',), ('मिलानमाथि',), ('ब्याजकान्तिपुर',), ('हुँदै',), ('चेल्सी',), ('विद्यालय',), ('भिडियोको',), ('वृद्धिपछि',), ('वर्षा',), ('रहरकान्तिपुर',), ('फाइनल',), ('लगाउने',), ('दिपिकाले',), ('लिगामा',), ('लय',), ('संघीय',), ('टिम',), ('उपाधि',), ('खेलहरूमा',), ('ग्रिनफिल्डले',), ('कथा',), ('चुकेको',), ('पठाउने',), ('सिराजको',), ('जित्यो',), ('संवाददाताफेसनसम्बन्धी',), ('सिराजले',), ('सञ्चालन',), ('यससँगै',), ('हराए',), ('फोरममा',), ('नेपालस्थित',), ('यती',), ('पुरानै',), ('संघ',), ('ब्याजदरको',), ('विजय',), ('गीता',), ('इन्टरनेसनलले',), ('लिगमा',), ('संवाददाताआईईसी',), ('सच्चा',), ('जनाएको',), ('विश्वकपका',), ('कलेज',), ('शतप्रतिशत',), ('असहमति',), ('बक्सिङमा',), (

In [9]:
df.head()

,Input,Category,Cleaned_Text,Tokenized_Text,No_Stopwords,Content_No_Stopwords,one_gram,bi_gram,tri_gram
0,"डलरको भाउ हालसम्मकै उच्च, किन बढिरहेको छ भाउ ?",business,डलरको भाउ हालसम्मकै उच्च किन बढिरहेको छ भाउ,"[डलरको, भाउ, हालसम्मकै, उच्च, किन, बढिरहेको, छ...","[डलरको, भाउ, हालसम्मकै, उच्च, बढिरहेको, भाउ]",डलरको भाउ हालसम्मकै उच्च बढिरहेको भाउ,"[(डलरको,), (भाउ,), (हालसम्मकै,), (उच्च,), (बढि...","[(डलरको, भाउ), (भाउ, हालसम्मकै), (हालसम्मकै, उ...","[(डलरको, भाउ, हालसम्मकै), (भाउ, हालसम्मकै, उच्..."
1,१२ वर्षमा २८ हजार मेगावाट बिजुली उत्पादनको लक्ष्य,business,वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य,"[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्...","[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्...",वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य,"[(वर्षमा,), (हजार,), (मेगावाट,), (बिजुली,), (उ...","[(वर्षमा, हजार), (हजार, मेगावाट), (मेगावाट, बि...","[(वर्षमा, हजार, मेगावाट), (हजार, मेगावाट, बिजु..."
2,भूतप्रभावी करविरुद्ध १६ बैंक अदालतमा,business,भूतप्रभावी करविरुद्ध बैंक अदालतमा,"[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]","[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]",भूतप्रभावी करविरुद्ध बैंक अदालतमा,"[(भूतप्रभावी,), (करविरुद्ध,), (बैंक,), (अदालतम...","[(भूतप्रभावी, करविरुद्ध), (करविरुद्ध, बैंक), (...","[(भूतप्रभावी, करविरुद्ध, बैंक), (करविरुद्ध, बै..."
3,तेस्रो देशका पर्यटकलाई भारतको ‘अघोषित नाकाबन्दी’,business,तेस्रो देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी,"[तेस्रो, देशका, पर्यटकलाई, भारतको, अघोषित, नाक...","[देशका, पर्यटकलाई, भारतको, अघोषित, नाकाबन्दी]",देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी,"[(देशका,), (पर्यटकलाई,), (भारतको,), (अघोषित,),...","[(देशका, पर्यटकलाई), (पर्यटकलाई, भारतको), (भार...","[(देशका, पर्यटकलाई, भारतको), (पर्यटकलाई, भारतक..."
4,साउन र भदौमा सरकारले उठायो कुल १ खर्ब ४१ अर्ब ...,business,साउन र भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व,"[साउन, र, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर...","[साउन, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर्ब,...",साउन भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व,"[(साउन,), (भदौमा,), (सरकारले,), (उठायो,), (कुल...","[(साउन, भदौमा), (भदौमा, सरकारले), (सरकारले, उठ...","[(साउन, भदौमा, सरकारले), (भदौमा, सरकारले, उठाय..."


In [10]:
done('Task 2')

Task 2: 2023-12-19 11:52:13.499840


## Task 3: Compute TF-IDF vectors for each vocabulary

TF-IDF stands for "Term Frequency-Inverse Document Frequency." TF-IDF vectors are numerical representations of text documents that capture the importance of individual words (terms) in the context of a collection of documents (corpus).
Here we calculate TF-IDF vectors for each vocabulary using Python's scikit-learn library's TfidfVectorizer function.

In [11]:
#COMPUTE TF - IDF VECTORS FOR ONE GRAM, BIGRAM AND TRIGRAM VOCABLARIES

#Compute TF-IDF vectors for each vocabulary
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer for one-gram, bi-gram, tri-gram vocabulary
tfidf_vectorizer_onegram = TfidfVectorizer(ngram_range=(1, 1))
tfidf_vectorizer_bigram = TfidfVectorizer(ngram_range=(2, 2))
tfidf_vectorizer_trigram = TfidfVectorizer(ngram_range=(3, 3))

# Fit and transform the text data for one-gram, bi-gram, tri-gram vocabulary
tfidf_vectors_onegram = tfidf_vectorizer_onegram.fit_transform(df['Content_No_Stopwords'])
tfidf_vectors_bigram = tfidf_vectorizer_bigram.fit_transform(df['Content_No_Stopwords'])
tfidf_vectors_trigram = tfidf_vectorizer_trigram.fit_transform(df['Content_No_Stopwords'])

# Convert TF-IDF vectors to a DataFrame for one-gram, bi-gram, tri-gram vocabulary
tfidf_df_onegram = pd.DataFrame(tfidf_vectors_onegram.toarray(), columns=tfidf_vectorizer_onegram.get_feature_names_out())
tfidf_df_bigram = pd.DataFrame(tfidf_vectors_bigram.toarray(), columns=tfidf_vectorizer_bigram.get_feature_names_out())
tfidf_df_trigram = pd.DataFrame(tfidf_vectors_trigram.toarray(), columns=tfidf_vectorizer_trigram.get_feature_names_out())

# Print the TF-IDF DataFrames for each vocabulary
print("TF-IDF DataFrame for One-gram Vocabulary:")
print(tfidf_df_onegram.head())

print("\nTF-IDF DataFrame for Bi-gram Vocabulary:")
print(tfidf_df_bigram.head())

print("\nTF-IDF DataFrame for Tri-gram Vocabulary:")
print(tfidf_df_trigram.head())



TF-IDF DataFrame for One-gram Vocabulary:
         अघ   अट        अद   अन  अपर   अफ  अबक   अभ   अम        अर  ...  सहमत  \
0  0.000000  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  ...   0.0   
1  0.000000  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  ...   0.0   
2  0.000000  0.0  0.456259  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  ...   0.0   
3  0.466544  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  ...   0.0   
4  0.000000  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.347147  ...   0.0   

         हज   हट   हण   हत  हतएज   हम   हर   हल  हलम  
0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  
1  0.469862  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  
2  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  
3  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  
4  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  

[5 rows x 367 columns]

TF-IDF DataFrame for Bi-gram Vocabulary:
      अघ बन  अट इल  अट दद  अट रप  अट सक  अट हज  अद लतम  अन आइतब  अन खड  अन 

In [12]:
df.head()

,Input,Category,Cleaned_Text,Tokenized_Text,No_Stopwords,Content_No_Stopwords,one_gram,bi_gram,tri_gram
0,"डलरको भाउ हालसम्मकै उच्च, किन बढिरहेको छ भाउ ?",business,डलरको भाउ हालसम्मकै उच्च किन बढिरहेको छ भाउ,"[डलरको, भाउ, हालसम्मकै, उच्च, किन, बढिरहेको, छ...","[डलरको, भाउ, हालसम्मकै, उच्च, बढिरहेको, भाउ]",डलरको भाउ हालसम्मकै उच्च बढिरहेको भाउ,"[(डलरको,), (भाउ,), (हालसम्मकै,), (उच्च,), (बढि...","[(डलरको, भाउ), (भाउ, हालसम्मकै), (हालसम्मकै, उ...","[(डलरको, भाउ, हालसम्मकै), (भाउ, हालसम्मकै, उच्..."
1,१२ वर्षमा २८ हजार मेगावाट बिजुली उत्पादनको लक्ष्य,business,वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य,"[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्...","[वर्षमा, हजार, मेगावाट, बिजुली, उत्पादनको, लक्...",वर्षमा हजार मेगावाट बिजुली उत्पादनको लक्ष्य,"[(वर्षमा,), (हजार,), (मेगावाट,), (बिजुली,), (उ...","[(वर्षमा, हजार), (हजार, मेगावाट), (मेगावाट, बि...","[(वर्षमा, हजार, मेगावाट), (हजार, मेगावाट, बिजु..."
2,भूतप्रभावी करविरुद्ध १६ बैंक अदालतमा,business,भूतप्रभावी करविरुद्ध बैंक अदालतमा,"[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]","[भूतप्रभावी, करविरुद्ध, बैंक, अदालतमा]",भूतप्रभावी करविरुद्ध बैंक अदालतमा,"[(भूतप्रभावी,), (करविरुद्ध,), (बैंक,), (अदालतम...","[(भूतप्रभावी, करविरुद्ध), (करविरुद्ध, बैंक), (...","[(भूतप्रभावी, करविरुद्ध, बैंक), (करविरुद्ध, बै..."
3,तेस्रो देशका पर्यटकलाई भारतको ‘अघोषित नाकाबन्दी’,business,तेस्रो देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी,"[तेस्रो, देशका, पर्यटकलाई, भारतको, अघोषित, नाक...","[देशका, पर्यटकलाई, भारतको, अघोषित, नाकाबन्दी]",देशका पर्यटकलाई भारतको अघोषित नाकाबन्दी,"[(देशका,), (पर्यटकलाई,), (भारतको,), (अघोषित,),...","[(देशका, पर्यटकलाई), (पर्यटकलाई, भारतको), (भार...","[(देशका, पर्यटकलाई, भारतको), (पर्यटकलाई, भारतक..."
4,साउन र भदौमा सरकारले उठायो कुल १ खर्ब ४१ अर्ब ...,business,साउन र भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व,"[साउन, र, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर...","[साउन, भदौमा, सरकारले, उठायो, कुल, खर्ब, अर्ब,...",साउन भदौमा सरकारले उठायो कुल खर्ब अर्ब राजस्व,"[(साउन,), (भदौमा,), (सरकारले,), (उठायो,), (कुल...","[(साउन, भदौमा), (भदौमा, सरकारले), (सरकारले, उठ...","[(साउन, भदौमा, सरकारले), (भदौमा, सरकारले, उठाय..."


In [13]:
done('Task 3')

Task 3: 2023-12-19 11:52:26.682212


## Task 4: Compute document similarity matrix
If your document size = N , this will result in the NxN matrix) for each vocab list

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity matrices for each vocabulary
cosine_sim_onegram = cosine_similarity(tfidf_vectors_onegram)
cosine_sim_bigram = cosine_similarity(tfidf_vectors_bigram)
cosine_sim_trigram = cosine_similarity(tfidf_vectors_trigram)

# Convert the similarity matrices to DataFrames
cosine_sim_onegram_df = pd.DataFrame(cosine_sim_onegram, columns=df.index, index=df.index)
cosine_sim_bigram_df = pd.DataFrame(cosine_sim_bigram, columns=df.index, index=df.index)
cosine_sim_trigram_df = pd.DataFrame(cosine_sim_trigram, columns=df.index, index=df.index)

cosine_sim_onegram_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.0,0.0,0.0,0.0,0.004365,0.142222,0.117344,0.046351,0.006400,...,0.007612,0.005365,0.007513,0.007196,0.008025,0.005897,0.008174,0.060602,0.056709,0.009767
1,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.022674,0.142655,0.000000,...,0.037803,0.000000,0.037310,0.000000,0.000000,0.117135,0.041254,0.061999,0.019339,0.121656
2,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,1.0,0.0,0.027729,0.000000,0.014271,0.000000,0.038596,...,0.000000,0.167707,0.045308,0.000000,0.000000,0.000000,0.000000,0.078042,0.024343,0.148829
4,0.0,0.0,0.0,0.0,1.0,0.258200,0.000000,0.140846,0.038711,0.214008,...,0.000000,0.058500,0.000000,0.000000,0.000000,0.032148,0.000000,0.000000,0.041782,0.000000


In [15]:
cosine_sim_bigram_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.127217,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.03368,0.000000,0.043144,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
cosine_sim_trigram_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
done('Task 4')

Task 4: 2023-12-19 11:52:36.141548


## Task 5: Write your interpretation on the result of Task 4.

The resulting cosine similarity matrices (cosine_sim_onegram, cosine_sim_bigram, and cosine_sim_trigram) represent the similarity between documents based on the one-gram, bi-gram, and tri-gram TF-IDF representations, respectively.

In [18]:
done('Task 5')

Task 5: 2023-12-19 11:52:38.426867
